# Stage 2: Filter by Metadata & Process Points

Given the geojsons created in step 1, process these into new geojson files with point data.

(This process is moved over here because node allows us to use turf)

In [1]:
var turf = require('turf')
var fs = require('fs')
var _ = require('lodash')

undefined

In [2]:
var config = JSON.parse(fs.readFileSync('./tweets-on-a-map.config'))
config

{ client: 'epic-analytics.cs.colorado.edu',
  database: 'matthew',
  collection: 'tweets',
  web_root: '/data/www/jennings/infovis' }

In [3]:
var fc = JSON.parse(fs.readFileSync(config.web_root + "/image-tweets-by-polygon.geojson"))
fc.features.length;

4525

In [4]:
var newFeatures = []
var count       = 0
var zeroes      = []
var zero_feats  = 0;
var tweets      = 0;
var dismissed   = 0;
var dismissedTweets = 0;

fc.features.forEach(function(feat){
    var area = Math.floor(turf.area(feat.geometry)/1000000) //Area is in square kilometers
    
    if (area < 9999999){ //Empirically chosen as a limit because otherwise the areas don't make sense :) 
        if (area==0){
            //We have a point geometry
            zeroes.push(feat);
            zero_feats += feat.properties.tweets.length
        }else{
            count++;
            feat.properties.area = Math.floor(area)
            newFeatures.push(feat)
            tweets+= feat.properties.tweets.length            
        }
    }else{
        dismissed++;
        dismissedTweets += feat.properties.tweets.length
    }
})
console.log("Features kept: " + count +", tweets: " + tweets + ", dismissed: " + dismissed + " with " + dismissedTweets + " tweets")
console.log("Zeroes counted " + zeroes.length +", tweets: " + zero_feats);
null

Features kept: 3563, tweets: 22582, dismissed: 4 with 78 tweets
Zeroes counted 958, tweets: 1129


null

Write these features to disk

In [5]:
var newFC = {type: "FeatureCollection",features: _.sortBy(newFeatures.concat(zeroes),function(f){return -f.properties.area})}
fs.writeFileSync(config.web_root + 'polygon-features.geojson', JSON.stringify(newFC))

"Wrote polygon geojson with " + newFC.features.length + " tweets."

'Wrote polygon geojson with 4521 tweets.'

## Create point geometries

In [6]:
newFC.features.forEach(function(feat){
    var center = turf.center(feat.geometry)
    feat.properties.poly = JSON.stringify(feat.geometry)
    feat.geometry = center.geometry
})
"Created " + newFC.features.length + " point geometries."

'Created 4521 point geometries.'

In [7]:
fs.writeFileSync(config.web_root + '/polygon-centers-with-tweets.geojson', JSON.stringify(newFC))
"Wrote point feature collection geojson without tweets."

'Wrote point feature collection geojson without tweets.'

In [8]:
newFC.features.forEach(function(feat){
    delete feat.properties.tweets
})
fs.writeFileSync(config.web_root + '/polygon-centers-no-tweets.geojson', JSON.stringify(newFC))
"Wrote point feature collection geojson with " + newFC.features.length + " tweets."

'Wrote point feature collection geojson with 4521 tweets.'